# Visualization Sandbox
A notebook to explore each of the vizualizations that will be included in the final summary report. This document is used to informally explore various visualization libraries and develop code to be implemented into the data visualization pipeline.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [2]:
df = pd.read_csv("../../dummy_cover_sheet_data.csv")